In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

In [6]:
# Loading Images and Labels
(train_ds, train_labels),(test_ds, test_labels)=tfds.load("tf_flowers",
                                                          split=["train[:70%]","train[:30%]"], # Train and Test Split,
                                                          batch_size=-1,as_supervised=True, # Include Labels
                                                          )

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [7]:
# Resizing images
train_ds= tf.image.resize(train_ds, (150,150))
test_ds= tf.image.resize(train_ds, (150,150))

In [8]:
# Transforming labels to correct format
train_labels= to_categorical(train_labels, num_classes=5)
test_labels= to_categorical(test_labels, num_classes=5)

In [9]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [10]:
# Loading VGG16 Model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False  ## Not Trainable Weights

58889256/58889256 [==============================] - 1s 0us/step


In [11]:
# Preprocess input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [12]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [14]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=5, validation_split=0.2, batch_size=64, callbacks=[es])

Epoch 1/5
33/33 [==============================] - 616s 19s/step - loss: 0.4249 - accuracy: 0.8433 - val_loss: 1.1862 - val_accuracy: 0.6362
Epoch 2/5
33/33 [==============================] - 563s 17s/step - loss: 0.3473 - accuracy: 0.8779 - val_loss: 1.4218 - val_accuracy: 0.6615
Epoch 3/5
33/33 [==============================] - 591s 18s/step - loss: 0.2259 - accuracy: 0.9178 - val_loss: 1.2639 - val_accuracy: 0.6868
Epoch 4/5
33/33 [==============================] - 590s 18s/step - loss: 0.1928 - accuracy: 0.9270 - val_loss: 1.3636 - val_accuracy: 0.6829
Epoch 5/5
33/33 [==============================] - 591s 18s/step - loss: 0.1356 - accuracy: 0.9582 - val_loss: 1.3485 - val_accuracy: 0.6946
